# Functional Programming Style and Streams

## Prime Number Check

An unoptimized naive algorithm.

In [1]:
function isPrime(n) {
  if (n < 2) {
    return false;
  }

  // 最简单的优化是对n开个根号
  for (let i = 2; i < n; i++) {
    if (n % i === 0) {
      return false;
    }
  }
  return true;
}

In [2]:
isPrime(2)

true

In [3]:
isPrime(11)

true

In [4]:
isPrime(2**13 - 1)

true

In [5]:
isPrime(2**11-1)

false

先用命令式风格实现两片代码。

## Sum of All Primes Under 100

In [6]:
let s = 0;
const n = 1000;
for (let i = 1; i <= n; i++) {
  if (isPrime(i)) {
    s += i;
  }
}
s

76127

## Find Three Prime Numbers Greater Than 1000

In [7]:
const list = [];
let start = 1000;
while (list.length < 3) {
  if (isPrime(start)) {
    list.push(start);
  }
  start++;
}

list

[ 1009, 1013, 1019 ]

Let's look at how to express these problems using functional programming.

## Functional Programming Utilities

In [8]:
function filter(f, ls) {
 return ls.filter(f)
}

function range(n) {
  return [...Array(n).keys()]
}

function sum(ls) {
  return ls.reduce((a, b) => a + b, 0)
}

## Calculate the Sum of All Primes Below 1000

In [9]:
sum(filter(isPrime, range(1000)))

76127

In [10]:
sum(filter(isPrime, range(2000)))

277050

How about applying this to multiple numbers? As usual, we need a function to wrap this logic.

## Again

In [11]:
const filter = f => ls => ls.filter(f)
const filterPrime = filter(isPrime)
const sumOfPrime = x => sum(filterPrime(x))

sumOfPrime(range(1000))
sumOfPrime(range(2000))

277050

## compose

Using `compose` to make it more explicit

In [12]:
const filter = f => ls => ls.filter(f)
const compose = (f, g) => x => f(g(x))

const sumOfPrime = compose(  // 函数的组合
  sum,                 // 2. 求和
  filter(isPrime),     // 1. 过滤出素数
)

sumOfPrime(range(1000))

76127

compose is right-associative, so the execution flow above is: filter first, then sum.

Think of it as data flowing from right to left.

```
output <<-- sum <<-- filter(isPrimce) <<-- input
```

Notice that when using compose with the curried form of filter, the data argument disappears - this is called pointfree style, but that's another story.

Let's move on to the second problem first.

## Find Three Prime Numbers Greater Than 1000

How can we solve this functionally?

Let's begin with a simple version, like this:

In [13]:
// take(3, filter(isPrime, range(1000, 2000)))

Let's extend the previous `range` function

In [14]:
function range([start, end]) {
  return Array(end - start).fill(0).map((_, i) => start + i)
}

In [15]:
range([1, 5])

[ 1, 2, 3, 4 ]

`take` is used to get the first elements from a list.

In [16]:
function take(n, list) {
  return list.slice(0, n)
}

In [17]:
take(3, filter(isPrime, range([1000, 2000])))

[ 1009, 1013, 1019 ]

## Problems

The above implementation has two issues:

- If we need 100 prime numbers greater than 1000, we don't know how large to set the upper bound of the range;
- Although we only need 3 numbers, it actually finds all prime numbers between 1000~2000, performing unnecessary calculations.

How can we express this in a functional style while also achieving lazy evaluation?
This "having our cake and eating it too" is very important.

Come to think of it, we need a "chain-like" structure where each element is "lazy" - this structure is called Stream.

## Natural Numbers

Let's begin exploring this structure with natural numbers, which start from 0 in modern set theory-based mathematics.

In [18]:
function N(n = 0) {
  return { value: n, next: () => N(n + 1) }
}

In [19]:
N()

{ value: 0, next: [Function: next] }

In [20]:
let o = N()
for (let i = 0; i < 10; i++) {
  console.log(o.value)
  o = o.next()
}

0
1
2
3
4
5
6
7
8
9


{ value: 10, next: [Function: next] }

Notice that it is immutable and pure.

### Refactoring

Note that the constructor (N) and next are actually the same thing, just from a different perspective.

In [21]:
const N = (n = 0) => {
  return { value: n, next: () => N(n + 1) }
}

let cur = N
for (let i = 0; i < 10; i++) {
  const iter = cur()  // 实例化元素， 即取次前调用。
    
  console.log(iter.value)
    
  cur = iter.next // 链式操作
}

0
1
2
3
4
5
6
7
8
9


[Function: next]

## Stream 

The above structure can be described in types as

```ts
type Stream<T> = () => StreamItem<T>
type StreamItem<T> = {
  value: T
  next: Stream<T>
}
```

Next, we'll implement some operations on this structure.

### take

In [22]:
// Number, Stream<T> -> [T]

function take(n, st) {
  const list = []
  let cur = st
  for (let i = 0; i < n && cur; i++) {
    const iter = cur()
    list.push(iter.value)
    cur = iter.next
  }
  return list
}

In [23]:
take(7, N)

[ 0, 1, 2, 3, 4, 5, 6 ]

### filter

In [24]:
function filter(f, st) {
  return () => {
    const iter = st()
    const next = iter.next ? filter(f, iter.next) : undefined
    if (f(iter.value)) {
      return { value: iter.value, next }
    }
    return next ? next() : undefined
  }
}

## Take the First 15 Even Numbers

In [25]:
const N = (n = 0) => {
  return { value: n, next: () => N(n + 1) }
}

take(15, filter(v => v % 2 === 0, N))

[ 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28 ]

It feels like time is being folded. Now let's look at prime.

### Take the First 10 Primes

In [26]:
take(10, filter(isPrime, N))

[ 2, 3, 5, 7, 11, 13, 17, 19, 23, 29 ]

## Take the First Three Primes Above 1000

In [27]:
take(3, filter(v => v > 1000 && isPrime(v), N))

[ 1009, 1013, 1019 ]

How to Find All Prime Numbers Between 1000 and 2000?

## Range

We need a stream that represents a Range.

In [28]:
function Range([a, b]) {
  const iter = value => () => {
    const next = value + 1 < b ? iter(value + 1) : undefined
    return { value, next }
  }
  return iter(a)
}

In [29]:
take(10000, filter(isPrime, Range([1000, 2000])))

[ 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117, 1123, 1129, 1151, 1153, 1163, 1171, 1181, 1187, 1193, 1201, 1213, 1217, 1223, 1229, 1231, 1237, 1249, 1259, 1277, 1279, 1283, 1289, 1291, 1297, 1301, 1303, 1307, 1319, 1321, 1327, 1361, 1367, 1373, 1381, 1399, 1409, 1423, 1427, 1429, 1433, 1439, 1447, 1451, 1453, 1459, 1471, 1481, 1483, 1487, 1489, 1493, 1499, 1511, 1523, 1531, 1543, 1549, 1553, 1559, 1567, 1571, 1579, 1583, 1597, 1601, 1607, 1609, 1613, 1619, 1621, 1627, 1637, 1657, 1663, 1667, 1669, 1693, 1697, 1699, 1709, 1721, 1723, 1733, 1741, 1747, 1753, 1759, 1777, 1783, 1787, 1789, 1801, 1811, 1823, 1831, 1847, 1861, 1867, 1871, 1873, 1877, 1879, 1889, 1901, 1907, 1913, 1931, 1933, 1949, 1951, 1973, 1979, 1987, 1993, 1997, 1999 ]

## Map

`map` for Stream

In [30]:
function map(f, st) {
  return () => {
    const iter = st()
    const value = f(iter.value)
    const next = iter.next ? map(f, iter.next) : undefined
    return { value, next }
  }
}

In [31]:
// 偶数
const _2N = map(x => x * 2, N)

take(10, _2N)

[ 0, 2, 4, 6, 8, 10, 12, 14, 16, 18 ]